In [6]:
import numpy as np
import madmom
import pickle
from madmom.ml.nn import NeuralNetwork
from copy import deepcopy

In [17]:
def _cnn_onset_processor_pad(data):
    """Pad the data by repeating the first and last frame 7 times."""
    pad_start = np.repeat(data[:1], 7, axis=0)
    pad_stop = np.repeat(data[-1:], 7, axis=0)
    return np.concatenate((pad_start, data, pad_stop))

In [7]:
with open('datasets/madmom_models-master/onsets/2013/onsets_cnn.pkl', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    base_model = u.load()

In [8]:
'''
 'bias',
 'pad',
 'reset',
 'stride',
 'weights']
'''
lay = base_model.layers[1]

linear_fn = base_model.layers[0].activation_fn

print(lay.bias.shape)
print(lay.weights.shape)
print(lay.activation_fn)
print(lay.pad)

(10,)
(3, 10, 7, 3)
<function tanh at 0x7fd2dcd508b0>
valid


In [9]:
lay.bias = np.zeros_like(lay.bias)
lay.weights = np.ones_like(lay.weights)/np.size(lay.weights)
lay.activation_fn = linear_fn

In [10]:

base_model.layers = [lay]
with open('models/toy_model.pickle', 'wb') as file_pi:
    pickle.dump(base_model, file_pi)

In [11]:
nn = NeuralNetwork.load('models/toy_model.pickle')


In [18]:
mm_proc_frames = np.load("results/processor_test/prep.npy")
mm_proc_frames = _cnn_onset_processor_pad(mm_proc_frames)
inv_std = np.load("models/bock2013pret_inv_std.npy")
mean = np.load("models/bock2013pret_mean.npy")
mm_frames_normalized = (mm_proc_frames - np.reshape(mean, (1,80,3)))*np.reshape(inv_std, (1,80,3))
mm_frames_normalized.min(), mm_frames_normalized.max()

(-1.1668240167396324, 0.9492300645795243)

In [19]:
act = nn(mm_frames_normalized)

In [20]:
print(act.mean())

0.05692679
